Adapted from https://www.eol.ucar.edu/content/sonic-tilt-corrections

In [136]:
sys.path.append('/home/elilouis/sublimationofsnow')
import sosutils
import xarray as xr
import numpy as np
from sklearn import linear_model
import altair as alt

# Download multiple daily datasets and combine

In [137]:
sos_download_dir='/data2/elilouis/sublimationofsnow/sosnoqc'
DATE_FORMAT_STR = '%Y%m%d'
datelist = ['20221215', '20221216', '20221217', '20221218', '20221219']
start_date = datelist[0]
end_date = datelist[-1]


VARIABLE_NAMES = [  
    'u_1m_c',   'v_1m_c',   'w_1m_c'
]

datasets = [xr.open_dataset(sosutils.download_sos_data_day(date, sos_download_dir))[VARIABLE_NAMES] for date in datelist]

sos_ds = sosutils.merge_datasets_with_different_variables(datasets, dim='time')

Fit, using least-squares, the averaged wind components to the equation of the plane,

$ w = a + bu + cv $

or

$ a = -bu - cv + w$

In [138]:
X_data = np.array([sos_ds['u_1m_c'], sos_ds['v_1m_c']]).reshape((-1,2))
Y_data = np.array(sos_ds['w_1m_c'])

In [139]:
reg = linear_model.LinearRegression().fit(X_data, Y_data)
a = reg.intercept_
b,c = reg.coef_
print("value of intercept, a:", a)
print("coefficients of equation of plane, (b, c): ", (b,c))

value of intercept, a: -0.106430694
coefficients of equation of plane, (b, c):  (0.0007045424, -0.005320101)


The sonic coordinates are defined

$U_s = \langle 1, 0, 0\rangle$

$V_s = \langle 0, 1, 0\rangle$

$W_s = \langle 0, 0, 1\rangle$

In [140]:
U_s = np.array([1,0,0])
V_s = np.array([0,1,0])
W_s = np.array([0,0,1])

We seek to define a new set of coordinates, streamwise coordinates, 

$U_f$

$V_f$

$W_f$

The normal vector to the plane, Wf, is

$W_f = \langle -b, -c, 1 \rangle / \sqrt{b^2 + c^2 + 1}$

which can be defined in terms of two angles,

$ tilt = arctan(\sqrt{b^2 + c^2}) $

where $tilt$ is the angle between the vertical sonic axis, $W_s$, and $W_f$, and should be positive and between 0 and 90˚, and

$ tiltaz = arctan(-c/-b) $

where $tiltaz$ is the azimuth of the projection of the flow normal vector in the sonic U-V plane, measured counterclockwise from $U_s$, one of the sonic axes.

$ Wf = ( sin(tilt)*cos(tiltaz), sin(tilt)*sin(tiltaz), cos(tilt) ) $


**For python, make sure to use atan2(-c,-b) to calculate $tiltaz$** to ensure trig functions return full range of -$\pi$ to $\pi$.

In [141]:
W_f = np.array([-b, -c, 1]) / np.sqrt( b**2 + c**2 + 1**2 )

In [142]:
tilt = np.arctan(np.sqrt(b**2 + c**2))
tiltaz = np.arctan2(-c, -b)

np.rad2deg(tilt), np.rad2deg(tiltaz)

(0.3074776935059586, 97.5438)

Check that these two angles correctly define $W_f$ (floating point errors are ok)

In [143]:
W_f - ( np.sin(tilt)*np.cos(tiltaz), np.sin(tilt)*np.sin(tiltaz), np.cos(tilt) )

array([ 3.50852783e-10, -9.14308785e-11, -1.11022302e-16])

$U_f$ can be defined as the intersection of the plane of mean flow with the plane defined by $U_s$ and $W_f$. 

$ U_{f, normal} = (W_f \text{ x } U_s) \text{ x } W_f $

$ U_f = U_{f, normal} / | U_{f, normal} |$

(This is one of two options offered, we use the second option offered and preferred by the source material).

Finally,

$V_f = W_f \text{ x } U_f$

In [144]:
U_f_normal_vector = np.cross(np.cross(W_f, U_s), W_f)
U_f = U_f_normal_vector / np.sqrt((U_f_normal_vector**2).sum())
V_f = np.cross(W_f, U_f)

For a wind vector in $\langle u, v, w \rangle$ sonic coordinates, with a $w$ offset "$a$", the corrected vector in mean flow coordinates $\langle u_f, v_f, w_f \rangle$ is computed by projecting the old velocity vectors onto the new axes.

$ u_f = U_f \cdot \langle u, v, w-a \rangle$

$ v_f = V_f \cdot \langle u, v, w-a \rangle$

$ w_f = W_f \cdot \langle u, v, w-a \rangle$

In [145]:
sos_ds

<xarray.Dataset>
Dimensions:  (time: 1440)
Coordinates:
  * time     (time) datetime64[ns] 2022-12-15T00:02:30 ... 2022-12-19T23:57:30
Data variables:
    u_1m_c   (time) float32 2.838 3.505 3.484 3.484 ... 0.8398 0.9758 0.4193
    v_1m_c   (time) float32 -2.653 -2.95 -2.402 -2.704 ... -1.118 -1.404 -1.708
    w_1m_c   (time) float32 -0.1893 -0.1837 -0.1427 ... -0.075 -0.09848 -0.1258
Attributes:
    history:                   Created: 2022-12-15 00:05:13 +0000\n
    NIDAS_version:             v1.2-1692
    calibration_file_path:     /h/eol/isfs/isfs/projects/SOS/ISFS/cal_files/n...
    project_config:            /h/eol/isfs/isfs/projects/SOS/ISFS/config/sos....
    wind3d_horiz_coordinates:  geographic
    file_length_seconds:       86400
    wind3d_horiz_rotation:     1
    wind3d_tilt_correction:    0

In [146]:
sos_ds = sos_ds.assign(
    u_1m_c_streamwise = (['time'], np.dot(U_f, np.array([sos_ds['u_1m_c'], sos_ds['v_1m_c'], sos_ds['w_1m_c'] - a]))),
    v_1m_c_streamwise = (['time'], np.dot(V_f, np.array([sos_ds['u_1m_c'], sos_ds['v_1m_c'], sos_ds['w_1m_c'] - a]))),
    w_1m_c_streamwise = (['time'], np.dot(W_f, np.array([sos_ds['u_1m_c'], sos_ds['v_1m_c'], sos_ds['w_1m_c'] - a]))),
)

In [165]:
# sos_df = sos_ds.sel(time=slice('2022-12-16 00:00', '2022-12-17 00:00')).to_dataframe().reset_index()
sos_df = sos_ds.to_dataframe().reset_index()

sos_df = sos_df.melt('time', [
    'u_1m_c',
    'v_1m_c',
    'w_1m_c',
    'u_1m_c_streamwise',
    'v_1m_c_streamwise',
    'w_1m_c_streamwise'  
])

sos_df['direction'] = sos_df['variable'].apply(lambda x: x[0])
sos_df['coordinate'] = sos_df['variable'].apply(lambda x: 'Mean Flow' if 'streamwise' in x else 'Sonic')

In [157]:
sos_df = sosutils.modify_df_timezone(sos_df, 'UTC', 'US/Mountain')

In [166]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [170]:
alt.Chart(sos_df).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('coordinate:N'),
).properties(width=1800, height = 150).facet(
    row = 'direction:N'
).resolve_scale(y='independent').display(renderer='svg')

alt.FacetChart(...)